In [ ]:
#importing packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import geopandas as gpd #having some dificulty with geopandas, have tried a number of different versions,
#appears pyproj is the issue
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from sklearn.neighbors import KNNClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

Data imported from Github repo, also available at the following source as provide by DrivenData on behalf of Taarifa (Tanzania Ministry of Water)

Data Source <https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/data/>

Data inlcudes a set of training data with labels and a set of test data for prediction and submission

Taking a look at the data below gives us a feel for what has been provided

In [ ]:
values = pd.read_csv('Source_data/trainset_values.csv')

In [ ]:
values.head()

In [ ]:
values.info()

We see that ther are some null values in columns 
['funder', 'installer', 'subvillage', 'public_meeting', 'scheme_management', 'scheme_name', 'permit']
that will need to be handled. It also appears that some columns provide overlapping information (extraction, management, payment, water quality and quantity, source and waterpoint). We will have to investiagte further if each column provides enough new information to keep. 

In [ ]:
classes = pd.read_csv('Source_data/trainset_labels.csv')


Taking a look at the training labels below we see that there are 3 groups: functional, non functional and 

In [ ]:
classes.head()

In [ ]:
sns.histplot(data=classes, x='status_group')
plt.show()

In [ ]:
values.columns

- amount_tsh - Total static head (amount water available to waterpoint)
- date_recorded - The date the row was entered
- funder - Who funded the well
- gps_height - Altitude of the well
- installer - Organization that installed the well
- longitude - GPS coordinate
- latitude - GPS coordinate
- wpt_name - Name of the waterpoint if there is one
- num_private -
- basin - Geographic water basin
- subvillage - Geographic location
- region - Geographic location
- region_code - Geographic location (coded)
- district_code - Geographic location (coded)
- lga - Geographic location
- ward - Geographic location
- population - Population around the well
- public_meeting - True/False
- recorded_by - Group entering this row of data
- scheme_management - Who operates the waterpoint
- scheme_name - Who operates the waterpoint
- permit - If the waterpoint is permitted
- construction_year - Year the waterpoint was constructed
- extraction_type - The kind of extraction the waterpoint uses
- extraction_type_group - The kind of extraction the waterpoint uses
- extraction_type_class - The kind of extraction the waterpoint uses
- management - How the waterpoint is managed
- management_group - How the waterpoint is managed
- payment - What the water costs
- payment_type - What the water costs
- water_quality - The quality of the water
- quality_group - The quality of the water
- quantity - The quantity of water
- quantity_group - The quantity of water
- source - The source of the water
- source_type - The source of the water
- source_class - The source of the water
- waterpoint_type - The kind of waterpoint
- waterpoint_type_group - The kind of waterpoint

We can add the labels onto the values dataframe in order to do some further EDA

In [ ]:
values.insert(loc=1, column='class', value=classes['status_group'])

In [ ]:
values.head()

Let's take a look at some details of the source columns ['source', 'source_type', 'source_class']

In [ ]:
values['source'].value_counts(), values['source_type'].value_counts(), values['source_class'].value_counts()

'source' and 'source_type' have very little difference expect:
- 'river' and 'lake' categories combined to 'river/lake'
- 'other' and 'unknown' combined into a single category called 'other'
- 'machine dbw' and 'hand dtw' were combined into a single 'borehole' category

In [ ]:
#Let's take a look at the distribution of the labels overall
labelcount = values.groupby('class')['id'].count()
labelcount = pd.concat([labelcount, values.groupby('class')['id'].apply(lambda x:'{0:.1f}%'.format(x.count()/len(values['id'])*100))], axis=1)
labelcount.columns = ['count', 'percent']
labelcount

We can then compare this distrubtion against the distribution with each source type. If the value for the different sources are consistent with the average then it would mean we aren't necessary gather more data from having the most robust amount of columns. However, if there are differences then we would want to keep those differences intact for our model. We are looking to choose which of the source columns to use in order to limit colinearity in our data, though we could add this information back in at a later time if we think it could increase our model perfomance.

In [ ]:
#Reviewing counts of labels per each source
sourcecount = values.groupby(['class', 'source'])['id'].count().unstack()
sourcecount

In [ ]:
#USing our counts to determine percentges for simpler comparison against the total percentages found above
sourceper = sourcecount/sourcecount.sum()*100
sourceper

As we will be doing multiple comparisons between the categorical label values and the other categorical columns, lets create a new fuction that create a clean and informational visual.

In [ ]:
def Chi_sq_test(df, dependant, independant):
    #takes in the names of a dependant and independant variable (column), runs a chi squared test and then outputs 
    #a seaborn heatmap of the percent difference between the expected and actual values
    
    #create cotingency table
    count_table = df.groupby([dependant, independant])['id'].count().unstack()
    count_table.fillna(0, inplace=True)
    count_table = count_table.astype('int')
    
    #Chi Squared test is for only counts above 5, we are keeping the same ratio, but increasing min value to 5 in each column
    if count_table.isin(range(0,5)).any().any():
        for j in range(len(count_table.columns)):
            for i in range(len(count_table.index)):
                if count_table.iloc[i,j] < 1:
                    count_table.iloc[i,j] = 5
                    count_table.iloc[:,j] = count_table.iloc[:,j]*5
                elif count_table.iloc[i,j] <5:
                    count_table.iloc[:,j] = count_table.iloc[:,j]*(5/count_table.iloc[i,j])
    
    stat, p, dof, expected = chi2_contingency(count_table)
    
    #print test information
    print('P-Value = {}'.format(p))
    print('Chi Statistic = {}'.format(stat))
    print('Degrees of Freedom = {}'.format(dof))
    
    #caluclate and print heatmap
    plt.figure(figsize=(12,6))
    sns.heatmap(((count_table - expected) / count_table *100), annot=True, vmax=100, vmin=-100, fmt='.1f', 
                annot_kws={'rotation': 90}, cmap='viridis')
    plt.title('Percent Difference of Expected vs. Actual Classes per {}'.format(str.title(independant)))
    plt.show()

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(values, 'class', 'source')

The overall data had values of:
- functional 54.3%
- functional needs repair 7.3% 
- non functional 38.4%

Source categories hand dtw, other, river, shallow well, unknown

Source categories dam, lake, machine dbh, rainwater harvesting, and spring deviate from the overall numbers in varying ways. For example, dam had nearly a reversal of the percentages for functional and non functional making it a strong feature for recognizing non functional well points. Also while rainwater harvesting has a slightly higher to overall percent functional, it also has double the functional needs repair of the overall and will be a good indicator for the that category. We will keep the 'source' column and drop 'source_type' and 'source_class'.

In [ ]:
data = values.drop(columns=['source_type', 'source_class'])
data.columns

In [ ]:
#plotting source and label comparisons
plt.figure(figsize=(16, 10))
ax=sns.histplot(data=values, x='source', hue='class', multiple='dodge')
total = len(values['class'])
for p in ax.patches:
    height=p.get_height()
    ax.text(x=p.get_x()+(p.get_width()/2), y=height+0., s=height, ha='center')
plt.title('Class vs Source')
plt.tight_layout()
plt.show()

We will look at the other features that have multiple columns. For now we are looking to keep only one of each column to limit collinearity, though we can come back later if we feel they have value to add. 

In [ ]:
values['extraction_type'].value_counts(), values['extraction_type_group'].value_counts(), values['extraction_type_class'].value_counts()

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'extraction_type')

The overall data had values of:
- functional 54.3%
- functional needs repair 7.3% 
- non functional 38.4%

Most categories seem to deviate from the norm so we want to keep ans many as possible, however some have relatively small data asscoaited with them, i.e. other-mkulima/shinyanga. For that reason we are going to use the intermediate break down with the exception of keeping ksb and submersible seperate as well as cemo and climax seperate. These categories that are merged under the extraction_type_group column seem to have enough value to keep seperate. We will keep the extraction_type column, but map other swn 81, other-play pump, walimi and other-mkulima/shinyanga together into a single other-handpump group

In [ ]:
data = data.drop(columns=['extraction_type_group', 'extraction_type_class'])
data['extraction_type'].replace({'other - swn 81':'other-handpump',
                                 'other - play pump':'other-handpump', 
                                 'walimi':'other-handpump', 
                                 'other - mkulima/shinyanga':'other-handpump',
                                'swn 80':'swn_80',
                                 'nira/tanira':'nira-tanira',
                                'india mark ii':'india_mark_ii',
                                'india mark iii':'india_mark_iii',
                                'other - rope pump':'other-rope_pump',}, inplace=True)
data['extraction_type'].unique()

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'extraction_type')

We should also clean up some of the source names before we go further to eliminate erroneous spaces and symbols.

In [ ]:
data['source'].replace({'shallow well':'shallow_well',
                       'machine dbh':'machine_dbh',
                       'rainwater harvesting':'rainwater_harvesting',
                       'hand dtw':'hand_dtw'}, inplace=True)
data['source'].unique()

In [ ]:
#plotting extraction_type and label comparisons
plt.figure(figsize=(16, 10))
ax=sns.histplot(data=values, x='extraction_type', hue='class', multiple='dodge')
total = len(values['class'])
for p in ax.patches:
    height=p.get_height()
    ax.text(x=p.get_x()+(p.get_width()/2), y=height+0., s=height, ha='center')
plt.title('Class vs Extraction')
plt.tight_layout()
plt.show()

Now lets take a look at some of the location columns

In [ ]:
#
data[['region', 'region_code', 'subvillage']].nunique()

There are over 19k subvillages. It would be better to take the more common villages and group the less common villages into an 'other' categories. 

In [ ]:
data['subvillage'].value_counts().head(40)

There are a number of subvillages that are not names. We will need to find them and correct them.

In [ ]:
svshort = data.loc[data['subvillage'].str.len() <=3]
svshort['subvillage'].unique()

In [ ]:
svshort['subvillage'].nunique(), data['subvillage'].nunique()

Based on the amount of subvillages that don't have proper names and the amount of categories, we are going to group all of the subvillages with less than 200 well points into a single category of other. This allows any predictive power form the areas with more wells, which we would think are more populous and possibly more used/serviced, to remain without creating noise.

In [ ]:
counts = data['subvillage'].value_counts()
counts = counts.loc[counts >=200]
counts = list(counts.index)
data.loc[~data['subvillage'].isin(counts), 'subvillage'] = 'other'

In [ ]:
data['subvillage'].unique()

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'subvillage')

In [ ]:
data['region'].unique()

In [ ]:
data['region_code'].unique()

We will drop the region column and keep the region code column as region code has more categories. Without further outside research we can't determine the match up between the regions and the region codes and therefore can't confirm there collinearity.

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'region_code')

In [ ]:
data.drop(columns=['region'], inplace=True)
data.columns

In [ ]:
data.isnull().sum()

We have some null values in the dataframe. Based upon the value being in categorical columns, witrh the exception of public meeting and permit, we will fill in the value 'unknown'. For permit and public meeting we will fill in False, making the assumption that the data not being recorded makes it less likely the additoinal administrative steps were taken.

In [ ]:
data.fillna(inplace=True, value={'installer':'unknown','permit':False, 'funder':'unknown', 'public_meeting':False, 
                                 'scheme_management':'unknown', 'scheme_name':'unknown'})
data.info()


Let's take a further look at some of the geographic data in column lga

In [ ]:
data['lga'].value_counts()

In [ ]:
data['lga'].str.contains('Urban|urban|Rural|rural').sum()

Similar to what we did with subvillage, we will keep only the categories with large numbers of well points. Since only 1 category besides other has more than 200 well points we will modify this into a binary column for lga_Njombe. 

In [ ]:
data['lga_Njombe'] = data['lga'].replace({'Njombe':1})
data.loc[data['lga_Njombe']!=1, 'lga_Njombe'] = 0
data['lga_Njombe'].unique()

In [ ]:
data.drop(columns=['lga'], inplace=True)

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'lga_Njombe')

Let's take a look at the payment infomration in columns payemnt and payment_type

In [ ]:
data['payment'].value_counts(), values['payment_type'].value_counts()

It appears that the two columns were recording the same values, but that the payment_type column has many more non-other entries. We will drop the payment column to avoid collinearity.

Now we will look at the basin column to see if it needs any cleaning/modification.

In [ ]:
data.drop(columns=['payment'], inplace=True)

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'payment_type')

In [ ]:
data['basin'].value_counts()

We will leave the categories un altered as it seems to have a good distribution, but we will modify some of the strings to eliminate the /'s

In [ ]:
data['basin'].replace({'Ruvuma / Southern Coast':'Ruvuma-Southern_Coast',
                     'Wami / Ruvu':'Wami-Ruvu'}, inplace=True)
data['basin'].unique()

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'basin')

Lets' look into the dates recorded column to see if there are any trends. We will start with tranforming it into a datetime object and then extract the month and year into seperate columns for simpler viewing.

In [ ]:
data['date_recorded']= pd.to_datetime(values['date_recorded'])

In [ ]:
data['date_recorded'].describe(datetime_is_numeric=True)

In [ ]:
data['date_recorded'].dt.year.value_counts()

In [ ]:
data['year']=data['date_recorded'].dt.year
data['month']=data['date_recorded'].dt.month
data[['month', 'year']]

As we did before let's take a look at the percentages for each label by month to see how it matches up with the overall percentages.

In [ ]:
plt.figure(figsize=(16, 8))
ax=sns.histplot(data=data, x='month', hue='class', multiple='dodge')
for p in ax.patches:
    height=p.get_height()
    ax.text(x=p.get_x()+(p.get_width()/2)+.05, y=height+0.25, s=height, ha='center')
plt.title('Month of Year vs. Class')
plt.show()

There appears to be a some seasonality to when inspections are performed. This possibly coincides with wet and dry seasons in the region. Persons managing the wellpoints would typically have additional inspections before and during times of use to ensure limited down time during the peak demand. It would be expected that all types of wellpoints would have better water access during a wet season as groundwater levels rise, though we don't know how this becomes associated with demand as more non-wellpoint bodies of water are likely available at the same time. 

In [ ]:
#Reviewing counts of labels per each month
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'month')

There does appear to be a difference month to month in level of functioning equipment. Let's take a look further at a single year

In [ ]:
plt.figure(figsize=(16, 12))
ax=sns.histplot(data=data.loc[data['year']>2010], x='year', hue='class', multiple='dodge')
for p in ax.patches:
    height=p.get_height()
    ax.text(x=p.get_x()+(p.get_width()/2)+.05, y=height+0.25, s=height, ha='center')
plt.xticks([2011, 2012, 2013])
plt.title('Class Distribution per Year')
plt.show()

2011 and 2013 have simliar distributions of each class. 2012 appears to have a higher percentage of non functional class but we don't know if that was more a by product of the samller smaple size of inspections. Without knowing the reason for the considerable drop in inspections, we can't confidently say that there was a different distribution of classes.

In [ ]:
#Reviewing counts of labels per each year
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'year')

We can drop a few additional columns that don't give value to our predictive power. wpt_name, num_private and recorded_by are all arbitrary informatio that we don't expect to gain insight from. They will be dropped.

In [ ]:
data.drop(columns=['wpt_name', 'num_private', 'recorded_by'], inplace=True)

In [ ]:
data.info()

We can set the the permit and public meeting columns to be 0 and 1 instead of True and False to make them easier for model ingestion

In [ ]:
data['public_meeting'] = data['public_meeting'].map({True:1, False:0})

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'public_meeting')

In [ ]:
data['permit'] = data['permit'].map({True:1, False:0})

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'permit')

Let's take a look at the district code column and see if it differs greatly from the region_code column

In [ ]:
data['district_code'].nunique(), data['district_code'].unique()

In [ ]:
data.loc[data['district_code']==data['region_code']]['region_code'].count()

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'district_code')

Based on there being limited times where the district_code is the same as the region_code and that we don't have a better definition for either variable, we will keep both

Let's continue by looking at construction_year and transforming it into a years old column

In [ ]:
plt.figure(figsize=(16,8))
sns.histplot(data=data, x='construction_year', hue='class', multiple='dodge', bins=20)
plt.title('Class Distribution per Construction Year')
plt.show()

In [ ]:
data['construction_year'].describe()

As it is a float and seems to have many zeros we will have to make some corrections

In [ ]:
#lets look at how the data is spread out
data['construction_year'].value_counts()

In [ ]:
#looking just at the non zero data
data.loc[data['construction_year']>0,'construction_year'].describe()

We will use 1999 as a fill in date for the zeroes. This is based on the nonzero entries having a mean of ~1997 and a median of 2000. We will also convert it to a datetime object

In [ ]:
data['construction_year'] = values['construction_year']

In [ ]:
data['construction_year'].replace({0:1999}, inplace=True)
data['construction_year'] = pd.to_datetime(data['construction_year'], format='%Y')
data['construction_year']

In [ ]:
data['years_old'] = data['date_recorded'].dt.year - data['construction_year'].dt.year
data['years_old']

In [ ]:
data['years_old'].describe()

In [ ]:
data.loc[data['years_old']<0, 'years_old'] = 1

In [ ]:
plt.figure(figsize=(16,8))
sns.histplot(data=data, x='years_old', hue='class', multiple='dodge', bins=20)
plt.title('Class Distribution per Construction Year')
plt.show()

Let's take a look at quality, quantity and waterpoint type. As with the previous source and extraction columns we are looking to limit collinearity without giving away data that is useful for prediction

In [ ]:
data['water_quality'].value_counts(), data['quality_group'].value_counts()

In [ ]:
data['quantity'].value_counts(), data['quantity_group'].value_counts()

For quality, it would seem the only change from water quality to quality group is combining salty columns, fluoride columns and renaming soft to good. We will drop the quality group column. 

For quantity, there is diference between the columns so we will drop the quantity group column

In [ ]:
data.drop(columns=['quality_group', 'quantity_group'], inplace=True)

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'water_quality')

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'quantity')

In [ ]:
data['waterpoint_type'].value_counts(), data['waterpoint_type_group'].value_counts()

The only difference being the combination of communal standpipe columns in waterpoint type group. We will drop the waterpoint type group.

In [ ]:
data.drop(columns=['waterpoint_type_group'],inplace=True)

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'waterpoint_type')

Let's look into the management data. It is likely we will follow the same logic as with the subvillages where we keep the larger groups and part the smaller ones into bins.

In [ ]:
data['scheme_management'].value_counts(), data['scheme_name'].value_counts()

The two scheme columns have different enough infomration to keep both. The scheme management column with be left alone except for moving the 1 'None' value into the 'other' category. The scheme name column will need to have it's categories paired down.

In [ ]:
data['scheme_name'].value_counts().head(20)

In [ ]:
counts2 = data['scheme_name'].value_counts()
counts2 = counts2.loc[counts2 >=200]
counts2 = list(counts2.index)
data.loc[~data['scheme_name'].isin(counts2), 'scheme_name'] = 'other'

In [ ]:
data['scheme_name'].unique()

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'scheme_name')

In [ ]:
data['management'].value_counts(), data['management_group'].value_counts()

We will keep both columns unchanged. Let's take a look into the installer and funder columns

In [ ]:
data['funder'].value_counts(), data['installer'].value_counts()

As with our previous categorical columns, there are a lot of categpories that have very small counts. We will compbine them into a single other category and keep the categories with larger counts seperated.

In [ ]:
data['funder'].value_counts().head(50)

As with our previous categorical columns, there are a lot of categpories that have relatively small counts. We will combine them into a single other category and keep the categories with larger counts seperated. The number for seperation is a guess for the time being and we can come back and adjust it if we find this to be an important feature to the model.

In [ ]:
counts3 = data['funder'].value_counts()
counts3 = counts3.loc[counts3 >=500]
counts3 = list(counts3.index)
data.loc[~data['funder'].isin(counts3), 'funder'] = 'other'

In [ ]:
data['funder'].unique()

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'funder')

In [ ]:
data['installer'].value_counts().head(20)

As with our previous categorical columns, there are a lot of categpories that have relatively small counts. We will combine them into a single other category and keep the categories with larger counts seperated. The number for seperation is a guess for the time being and we can come back and adjust it if we find this to be an important feature to the model.

In [ ]:
counts4 = data['installer'].value_counts()
counts4 = counts4.loc[counts4 >=500]
counts4 = list(counts4.index)
data.loc[~data['installer'].isin(counts4), 'installer'] = 'other'

In [ ]:
data['installer'].unique()

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'installer')

We are curious to see if popultion plays a role in the distribution of classes. More population likely means more demand, but also could mean more resources in the area to fix problems more quickly and/or maintaint the wellpoint more frequently allowing for less issues leading the wellpoint being non functional

In [ ]:
data['population'].describe()

In [ ]:
plt.figure(figsize=(16,8))
sns.histplot(data=data, x='population', hue='class', bins=10, multiple='dodge')
plt.title('Class Distribution vs. Population')
plt.show()

Due to the long right tail of the data it is difficult to see where the distribution of the lower end data whihc includes the majority of the data. We can look at only areas with populations between 2 and 2000. This will also eliminate the assumedly erroneaus entries of 0 and 1 for population from the visual.

In [ ]:
data.loc[data['population'].between(2, 2000), 'population'].describe()

In [ ]:
plt.figure(figsize=(16,8))
sns.histplot(data=data.loc[data['population'].between(2,2000)], x='population', hue='class', bins=10, multiple='dodge')
plt.title('Class Distribution per Population, Population <= 2000')
plt.show()

We will create a new bin column for population with additional bins at the lower end.

In [ ]:
data['popbins'] = pd.cut(data['population'], [-1,2,250,500,1000,2500,10000,40000], labels=list(range(1,8)))

In [ ]:
data['popbins']

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'popbins')

Let's take a look at one of the final categorical features: ward. We can check if it matches up with the subvillage column in which case we will simply drop it to avoid colinearity. If not it is likely to be another column with many categories that will need to have categories with smaller counts combined.

In [ ]:
data['ward'].value_counts()

In [ ]:
(data['ward']==data['subvillage']).sum()

In [ ]:
data['ward'].value_counts().plot()
plt.title('Counts of Ward Categories')
plt.xlabel('Ward')
plt.ylabel('Count')
plt.show()

We can break the wards in five categories: verybig, big, medium, small and verysmall. If we find these are important features for the model then we can also come back and expand on the amount of categories.

In [ ]:
counts5 = data['ward'].value_counts()
verybig = counts5.loc[counts5.between(200,400)].index
big = counts5.loc[counts5.between(100,200)].index
medium = counts5.loc[counts5.between(50,100)].index
small = counts5.loc[counts5.between(25,50)].index
verysmall = counts5.loc[counts5 <=25].index
data.loc[data['ward'].isin(verybig), 'ward'] = 'verybig'
data.loc[data['ward'].isin(big), 'ward'] = 'big'
data.loc[data['ward'].isin(medium), 'ward'] = 'medium'
data.loc[data['ward'].isin(small), 'ward'] = 'small'
data.loc[data['ward'].isin(verysmall), 'ward'] = 'verysmall'

In [ ]:
data['ward'].unique()

In [ ]:
#Let's take a look at a comparison of classes and sources
Chi_sq_test(data, 'class', 'ward')

Let's take a look at the latitude and longitude columns. It is possible we will see this ditribution line up well with population as we expect more wellpoints in populated areas, though we aren't sure what the calss trend for these areas is yet.

In [ ]:
plt.figure(figsize=(16,16))
sns.scatterplot(data=data, y='latitude', x='longitude', hue='class')
plt.title('Latitude/Longitude Mapping')
plt.show()

There is a few datapoints that seem to have incorrect values, lets take a look.

In [ ]:
data['longitude'].describe()

In [ ]:
data.loc[data['longitude']==0]

The points all have the Lake Victoria basin. We will use an random value between the 25th and 75th percentiles from all datapoints in the Lake Victoria basin to fill in the zero values.

In [ ]:
data.loc[data['basin']=='Lake Victoria']['longitude'].describe()

In [ ]:
data.loc[data['longitude'] == 0, 'longitude'] = np.random.choice(range(31,33), 1812)

In [ ]:
plt.figure(figsize=(16,16))
sns.scatterplot(data=data, y='latitude', x='longitude', hue='class')
plt.title('Latitude/Longitude Mapping')
plt.show()

There is still an issue with a latitude value being zero, lets take a look

In [ ]:
#Finding all columns with latitude == 0
data.loc[data['latitude']==0]

In [ ]:
#widening the search
data.loc[data['latitude']>-1]

In [ ]:
#we can further tighen the search now that we see some of the issue values as -2.0E-08
data.loc[data['latitude']>-.01]

Let's fill these values in a similar manner to the longitiude values using a random value between the 25th and 75th percentile of Lake Victoria basin datapoints.

In [ ]:
data.loc[data['basin']=='Lake Victoria']['latitude'].describe()

In [ ]:
data.loc[data['latitude']>-0.01, 'latitude'] = -1*np.random.choice(range(1,2))

In [ ]:
plt.figure(figsize=(16,16))
sns.scatterplot(data=data, y='latitude', x='longitude', hue='class')
plt.title('Latitude/Longitude Mapping')
plt.show()

In [ ]:
plt.figure(figsize=(16,16))
sns.scatterplot(data=data, y='latitude', x='longitude', hue='gps_height')
plt.title('Latitude/Longitude vs. GPS Height')
plt.show()

In [ ]:
data['gps_height'].describe()

In [ ]:
data.loc[data['gps_height']>0, 'gps_height'].describe()

In [ ]:
lin_gps_trainX = data.loc[data['gps_height']>0, ['latitude', 'longitude']]
lin_gps_trainy = data.loc[data['gps_height']>0, ['gps_height']]
lin = LinearRegression()
lin.fit(lin_gps_trainX, lin_gps_trainy)
data.loc[data['gps_height']<=0, ['gps_height']] = lin.predict(data.loc[data['gps_height']<=0, ['latitude', 'longitude']])

In [ ]:
#data.loc[data['gps_height']<1, 'gps_height'] = np.random.choice(range(471, 1512), 59400-37466)

In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(data=data, x='gps_height', hue='class', multiple='dodge', bins=15, element='poly', fill=False)
plt.title('Class Distribution vs. GPS Height')
plt.show()

In [ ]:
data['amount_tsh'] = data['amount_tsh'].astype('int')
data['amount_tsh'].describe()

In [ ]:
data['amount_tsh'].value_counts()

In [ ]:
data.loc[data['amount_tsh']>2000, 'amount_tsh'].value_counts().head(10)

In [ ]:
data['amount_tsh'].sort_values().tail(20)

We are going to seperate this into bins to allow it function better with the classification algorithims. There also appears to be some issues with the top end values as 350,000 ft of head is more than 10x the height of Mt. Everest aka not possible for static head. We will use Standard Scaler on this column during feature engineering to ensure these high values don't cause the feature to be overweighted. We are also going to cap all values as 5000 and because more than two thirds of the column is at value zero, we will turn this into a categorical column of bins and eliminate the continuous column.

In [ ]:
data.loc[data['amount_tsh']>2000, 'amount_tsh'] = 2000

In [ ]:
data['amount_tsh']=values['amount_tsh']
data.loc[~data['amount_tsh'].between(1,4999), 'amount_tsh'].describe()

In [ ]:
#lin_tsh_trainX = data.loc[data['amount_tsh'].between(1,4999), ['gps_height', 'latitude', 'longitude']]
#lin_tsh_trainy = data.loc[data['amount_tsh'].between(1,4999), ['amount_tsh']]
lin_tsh = LinearRegression()
lin_tsh.fit(lin_tsh_trainX, lin_tsh_trainy)
lin_tsh.score(lin_tsh_trainX,lin_tsh_trainy)
#data.loc[~data['amount_tsh'].between(1,4999), 'amount_tsh'] = lin_tsh.predict(data.loc[~data['amount_tsh'].between(1,4999), ['gps_height', 'latitude', 'longitude']])

In [ ]:
data.loc[~data['amount_tsh'].between(1,4999), 'amount_tsh'].describe()

In [ ]:
sns.scatterplot(data=data, x='gps_height', y='amount_tsh')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
lin_tsh_trainX = data[['latitude', 'longitude']]
scaler = StandardScaler()
gps = scaler.fit_transform(data['gps_height'].values.reshape(-1,1))
lin_tsh_trainX.insert(1, value=gps, column='gps_height')
lin_tsh_trainX = lin_tsh_trainX.merge(pd.get_dummies(data[['waterpoint_type', 'quantity', 'extraction_type','source']]),
                                      left_index=True, right_index=True)
lin_tsh_trainX.insert(0, column = 'tsh', value=data['amount_tsh'])
lin_tsh_trainX = lin_tsh_trainX.loc[lin_tsh_trainX['tsh'].between(1,4999)]
lin_tsh_trainX.drop(columns=['tsh'], inplace=True)
lin_tsh_trainy = data.loc[data['amount_tsh'].between(1,4999), ['amount_tsh']]
lin_tsh_trainX


In [ ]:
X, tX, y, ty =  train_test_split(lin_tsh_trainX, lin_tsh_trainy, test_size=0.25, random_state=42)
lin_tsh = ElasticNet(max_iter=5000)
rsearch = RandomizedSearchCV(lin_tsh, param_distributions={'alpha':[.1, .2, .3, .4, .5, .6, .7, .8, .9, 1], 
                                                           'l1_ratio':[0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1]},
                            scoring= 'neg_mean_absolute_error', cv=4)#', 'neg_mean_absolute_percentage_error', 
                                     #'neg_root_mean_squared_error', 'r2'], refit='r2', cv=4)

In [ ]:
rsearch.fit(X, y)

In [ ]:
rsearch.cv_results_

In [ ]:
rsearch.score(tX, ty)

In [ ]:
pred = rsearch.predict(tX)
pred

In [ ]:
lin_tsh_fillX = data[['latitude', 'longitude']]
scaler = StandardScaler()
gps = scaler.fit_transform(data['gps_height'].values.reshape(-1,1))
lin_tsh_fillX.insert(1, value=gps, column='gps_height')
lin_tsh_fillX = lin_tsh_fillX.merge(pd.get_dummies(data[['waterpoint_type', 'quantity', 'extraction_type','source']]),
                                      left_index=True, right_index=True)
lin_tsh_fillX.insert(0, column = 'tsh', value=data['amount_tsh'])
lin_tsh_fillX = lin_tsh_fillX.loc[~lin_tsh_trainX['tsh'].between(1,4999)]
lin_tsh_trainy = data.loc[data['amount_tsh'].between(1,4999), ['amount_tsh']]

In [ ]:
tsh_pred = rsearch.predict(lin_tsh_fillX)

In [ ]:
lin_tsh_trainX.columns

In [ ]:
data['tsh_bins'] = data['amount_tsh']

In [ ]:
data['tsh_bins'] = pd.qcut(data['tsh_bins'], q=5, duplicates='drop',labels=False)
data['tsh_bins']

In [ ]:
data['amount_tsh']=data['amount_tsh'].astype('int')

In [ ]:
data['amount_tsh'].describe()

Let's see visually if the ratio of functional to non functional seems to improve based on higher tsh

In [ ]:
plt.figure(figsize=(12,8))
sns.histplot(data = data[['amount_tsh', 'class']], x='amount_tsh', hue='class', multiple='dodge')
plt.title('Distribution of Classes by Static Head Pressure')
plt.show()

Not suprisingly the more head available the higher the percentage of wellpoints are functional. This makes sense when you consider that higher head pressures would be able to more easily push through mechancial issues like lack of lubrication as well as dirt build up in piping. We assuming that these are common cause of non functional class.

In [ ]:
data.drop(columns=['date_recorded', 'construction_year'], inplace=True)

In [ ]:
data.to_pickle('Data/train_values_EDA2.pkl')

While we have further EDA and cleaning that could be done we are going to move forward at this point to see how our model performs with what we currently have. We can always come back and ajust the number of bins for a category and or maxes and mins for continuos variables.

We can create a single function that will perform all of our adjustments above. This will allow it to be run on the submission values as well and guarantee we are using the same process.

In [ ]:
%%writefile Functions\model_transformer.py 
def model_transformer(model_data):
    #Applies transformations from EDA notebook to training and testing sets to ensure same changes are made
    #Correct names in extraction_type
    data = model_data
    
    data['extraction_type'].replace({'other - swn 81':'other-handpump',
                                 'other - play pump':'other-handpump', 
                                 'walimi':'other-handpump', 
                                 'other - mkulima/shinyanga':'other-handpump',
                                'swn 80':'swn_80',
                                 'nira/tanira':'nira-tanira',
                                'india mark ii':'india_mark_ii',
                                'india mark iii':'india_mark_iii',
                                'other - rope pump':'other-rope_pump',}, inplace=True)
    #correct names in source
    data['source'].replace({'shallow well':'shallow_well',
                       'machine dbh':'machine_dbh',
                       'rainwater harvesting':'rainwater_harvesting',
                       'hand dtw':'hand_dtw'}, inplace=True)

    #Group low count subvillages in other
    counts = data['subvillage'].value_counts()
    counts = counts.loc[counts >=200]
    counts = list(counts.index)
    data.loc[~data['subvillage'].isin(counts), 'subvillage'] = 'other'

    data.fillna(inplace=True, value={'installer':'unknown','permit':False, 'funder':'unknown', 'public_meeting':False, 
                                 'scheme_management':'unknown', 'scheme_name':'unknown'})

    #create and boolean lga_Njombe column
    data['lga_Njombe'] = data['lga'].replace({'Njombe':1})
    data.loc[data['lga_Njombe']!=1, 'lga_Njombe'] = 0
    data['lga_Njombe'] = data['lga_Njombe'].astype('int')

    #remove slashes from basin names
    data['basin'].replace({'Ruvuma / Southern Coast':'Ruvuma-Southern_Coast',
                     'Wami / Ruvu':'Wami-Ruvu'}, inplace=True)

    #convert date_recorded column to datetime object and edxtract month and year
    data['date_recorded']= pd.to_datetime(data['date_recorded'])
    data['date_recorded'].describe(datetime_is_numeric=True)
    data['year']=data['date_recorded'].dt.year
    data['month']=data['date_recorded'].dt.month

    ##Convert public_meeting column to 1 or 0
    data['public_meeting'] = data['public_meeting'].map({True:1, False:0})

    #Convert permit column to 1 or 0
    data['permit'] = data['permit'].map({True:1, False:0})

    #Correct construction_year with 1999, create years_old column
    data['construction_year'].replace({0:1999}, inplace=True)
    data['construction_year'] = pd.to_datetime(data['construction_year'], format='%Y')
    data['years_old'] = data['date_recorded'].dt.year - data['construction_year'].dt.year

    #Group low count scheme_names under other
    counts2 = data['scheme_name'].value_counts()
    counts2 = counts2.loc[counts2 >=200]
    counts2 = list(counts2.index)
    data.loc[~data['scheme_name'].isin(counts2), 'scheme_name'] = 'other'
    
    #group low count funders under other
    counts3 = data['funder'].value_counts()
    counts3 = counts3.loc[counts3 >=500]
    counts3 = list(counts3.index)
    data.loc[~data['funder'].isin(counts3), 'funder'] = 'other'
    data.loc[data['funder']=='Government Of Tanzania', 'funder'] = 'gov_tanz'

    #Group low count installers under other
    counts4 = data['installer'].value_counts()
    counts4 = counts4.loc[counts4 >=500]
    counts4 = list(counts4.index)
    data.loc[~data['installer'].isin(counts4), 'installer'] = 'other'
    
    #Create column for population bins
    data['popbins'] = pd.cut(data['population'], [-1,2,250,500,1000,2500,10000,40000], labels=list(range(1,8)))
    
    #Amount_TSH - Change to bins
    data.loc[data['amount_tsh']>5000, 'amount_tsh'] = 5000
    data.loc[data['amount_tsh']>0, 'amount_tsh'] = pd.qcut(data.loc[data['amount_tsh']>0, 'amount_tsh'], 
                                                           q=5, duplicates='drop',labels=False)           
    
    #Ward Feature - Change to Bins
    counts5 = data['ward'].value_counts()
    verybig = counts5.loc[counts5.between(200,400)].index
    big = counts5.loc[counts5.between(100,200)].index
    medium = counts5.loc[counts5.between(50,100)].index
    small = counts5.loc[counts5.between(25,50)].index
    verysmall = counts5.loc[counts5 <=25].index
    data.loc[data['ward'].isin(verybig), 'ward'] = 'verybig'
    data.loc[data['ward'].isin(big), 'ward'] = 'big'
    data.loc[data['ward'].isin(medium), 'ward'] = 'medium'
    data.loc[data['ward'].isin(small), 'ward'] = 'small'
    data.loc[data['ward'].isin(verysmall), 'ward'] = 'verysmall'
    
    #Latitude-Longitiude - Correct near zero values
    data.loc[data['longitude'] == 0, 'longitude'] = np.random.choice(range(31,33))
    data.loc[data['latitude']>-0.01, 'latitude'] = -1*np.random.choice(range(1,2))
    
    lin_gps_trainX = data.loc[data['gps_height']>0, ['latitude', 'longitude']]
    lin_gps_trainy = data.loc[data['gps_height']>0, ['gps_height']]
    lin = LinearRegression()
    lin.fit(lin_gps_trainX, lin_gps_trainy)
    data.loc[data['gps_height']<=0, ['gps_height']] = lin.predict(data.loc[data['gps_height']<=0, ['latitude', 'longitude']])
    
    data.drop(columns=['source_type', 'source_class', 'extraction_type_group', 'extraction_type_class', 
                       'region', 'wpt_name', 'num_private', 'recorded_by', 'quality_group', 'quantity_group',
                       'waterpoint_type_group', 'payment', 'construction_year', 'date_recorded', 'lga'], inplace=True)
    return data

In [ ]:
model_data = model_transformer(values)

In [ ]:
data.columns == model_data.columns

In [ ]:
model_data.to_pickle('Data/model_data.pkl')